# Chains

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Basics

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

ChatOpenAI model

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")

Define Prompt Templates

In [4]:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me {joke_count} jokes."),
]

In [5]:
prompt_template = ChatPromptTemplate.from_messages(messages)

Create the combined chain using LangChain Expression Language (LCEL)

In [7]:
chain = prompt_template | model
result = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(result)

content='Sure! Here are three lawyer jokes for you:\n\n1. Why don’t lawyers go to the beach?  \n   Because cats keep trying to bury them in the sand!\n\n2. What’s the difference between a lawyer and a herd of buffalo?  \n   The lawyer charges more!\n\n3. How many lawyer jokes are there, anyway?  \n   Only three. The rest are true stories! \n\nHope you got a good chuckle out of those!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 26, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-ef7fffed-16a8-4b4e-b585-08d5314c675b-0' usage_metadata={'input_tokens': 26, 'output_tokens': 91, 'total_tokens': 117, 'input_token_details

**StrOutputParser**: OutputParser that parses LLMResult into the top likely string.

In [8]:
chain = prompt_template | model | StrOutputParser()
result = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(result)

Sure, here are three lawyer-themed jokes for you:

1. Why don’t lawyers go to the beach?  
   Because cats keep trying to bury them in the sand!

2. What’s the difference between a lawyer and a herd of buffalo?  
   The lawyer charges more!

3. How can you tell when a lawyer is lying?  
   Their lips are moving—just like when they’re talking about their fees! 

Hope you enjoy them!


## Chains under the hood

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableSequence, RunnableLambda

Create a ChatOpenAI model

In [10]:
model = ChatOpenAI(model="gpt-4o-mini")

Define Prompt Template

In [11]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me {joke_count} jokes."),
])

Create individual runnables (steps in a chain)

- Runnable: Is a task
- RunnableLambda: Is a task that is a lambda function
- RunnableSequence: Is a sequence of tasks

All those tasks put together in a sequence is called **Runnable Seqeunce** or **Chain**

In [12]:
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

RunnableSequence (equivalent to the LCEL chain)

* first: single runnable
* middle: multiple runnables (pass in as a list)
* third: single runnable

In [13]:
chain = RunnableSequence(first = format_prompt,middle = [invoke_model], last=parse_output)

Run the chain

In [14]:
response = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(response)

Sure! Here are three lawyer-themed jokes for you:

1. Why don’t lawyers go to the beach?
   Because cats keep trying to bury them in the sand!

2. What's the difference between a lawyer and a herd of buffalo?
   The lawyer charges more!

3. How can you tell if a lawyer is lying?
   Their lips are moving!

Hope these gave you a laugh!


## Chains Extended

Create a ChatOpenAI model

In [15]:
model = ChatOpenAI(model="gpt-4o-mini")

Define Prompt Template

In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}"),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

Additional processing steps using RunnableLambda

In [17]:
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count:{len(x.split())}\n{x}")

Combined chain using LangChain Expression Language (LCEL)

In [18]:
chain = prompt_template | model | StrOutputParser()| uppercase_output | count_words

response = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(response)

Word count:65
SURE, HERE ARE THREE LAWYER-THEMED JOKES FOR YOU:

1. WHY DON'T LAWYERS PLAY HIDE AND SEEK?  
   BECAUSE GOOD LUCK HIDING WHEN EVERY TIME THEY FIND YOU, THEY’LL JUST BILL YOU BY THE HOUR!

2. WHAT'S THE DIFFERENCE BETWEEN A LAWYER AND A HERD OF BUFFALO?  
   THE LAWYER CHARGES MORE!

3. WHY DID THE LAWYER CROSS THE ROAD?  
   TO SUE THE CHICKEN ON THE OTHER SIDE!


## Chains Parallel

In [20]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain.schema.output_parser import StrOutputParser

Create a ChatOpenAI model

In [21]:
model = ChatOpenAI(model="gpt-4o-mini")

Define prompt template

In [22]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are an expert product reviewer"),
    ("human","List the main features of the product {product_name}")
])

Define pro analysis step

In [29]:
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages([
        ("system","You are an expert product reviewer"),
        ("human","Given these features : {features}, list the pros of these features")
    ])
    return pros_template.format_prompt(features=features)

Define cons analysis step

In [30]:
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer"),
            (
                "human",
                "Given these features : {features}, list the cons of these features",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)

Combine Pros and Cons into a final review

In [31]:
def combine_pros_cons(pros, cons):
    return f"Pros: {pros}\nCons: {cons}"

Simplify branches with Langchain Expression Language (LCEL)

In [32]:
pros_branch_chain = RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
cons_branch_chain = RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()

Combined chain using LangChain Expression Language (LCEL)

- Runnable that runs a mapping of Runnables in parallel, and returns a mapping of their outputs.
- RunnableParallel is one of the two main composition primitives for the LCEL, alongside RunnableSequence. It invokes Runnables concurrently, providing the same input to each
- For each key-value pair in the RunnableParallel branches, a runnable is invoked and executed

In [33]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(
        lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"])
    )
)

In [34]:
result = chain.invoke({"product_name": "MacBook Pro"})
print(result)

Pros: The MacBook Pro, as described with its features, has numerous advantages that cater to professionals and power users alike. Here are the pros associated with each of those features:

1. **Display**:
   - **Stunning Visuals**: The Retina display with True Tone technology enhances color accuracy and brightness, making it ideal for graphic design, photo editing, and content consumption.
   - **Exceptional Brightness**: Up to 1600 nits allows for excellent visibility in bright environments and boosts HDR content.
   - **Wide Color Gamut**: P3 color support ensures true-to-life colors, enhancing creative work.
   - **Smooth Scrolling**: ProMotion technology with a 120Hz refresh rate delivers an ultra-smooth experience, making tasks like scrolling and gaming more engaging.

2. **Chip**:
   - **Powerful Performance**: Apple Silicon (M1 Pro, M1 Max, M2) provides robust multi-core performance for demanding applications and workloads.
   - **Efficient Processing**: High-performance and eff

## Chains Branching

In [35]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from langchain.prompts import ChatPromptTemplate

Create a ChatOpenAI model

In [36]:
model = ChatOpenAI(model="gpt-4o-mini")

Prompt templates for different types of feedbacks

In [37]:
positive_feedback_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Generate a thank you note for this positive feedback: {feedback}")
])

negative_feedback_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Generate a response addressing this negative feedback: {feedback}")
])

neutral_feedback_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Generate a request for more details for this neutral feedback: {feedback}")
])

escalate_feedback_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Generate a message to escalate this to feedback to human agent: {feedback}")
])

Feedback Classification Template

In [38]:
classification_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Classify this feedback as positive,negative,neutral or escalate: {feedback}")
])

**Runnable branches for handling feedbacks**

- Runnable that selects which branch to run based on a condition.

- The Runnable is initialized with a list of (condition, Runnable) pairs anda default branch.

- When operating on an input, the first condition that evaluates to True is selected, and the corresponding Runnable is run on the input.

- If no condition evaluates to True, the default branch is run on the input.


In [39]:
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser()
)

Classification chain

In [40]:
classification_chain = classification_template | model | StrOutputParser()

Combine classification chain and response generation into one chain

In [41]:
chain = classification_chain | branches

In [42]:
review = "The product is terrible. It broke after just one use and quality is very poor"
result = chain.invoke({"feedback": review})
print(result)

**Response to Negative Feedback:**

Dear [Name],

Thank you for taking the time to share your thoughts with us. We genuinely appreciate your feedback, as it helps us identify areas where we can improve.

We are sorry to hear that your experience did not meet your expectations. It’s concerning to us that [specific issue] impacted your experience negatively. We strive to provide each of our customers with the best possible service, and it’s clear that we fell short in this instance.

We would love the opportunity to learn more about your experience and how we can make things right. If you're open to it, please reach out to us directly at [contact information]. Your insights are invaluable to us, and we want to ensure you feel heard and valued.

Thank you once again for your feedback. We hope to have the chance to improve your experience in the future.

Best regards,

[Your Name]  
[Your Position]  
[Company Name]  
[Contact Information]
